# Notes
- Replace nulls with appropriate values - DONE
- Deal with time variables - take delta between time variables instead of the variables - DONE
- Remove the single duplicate row - DONE
- Calculate delivered/alloted ratio for riders - DONE
- Include reassignment or not? - NO.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
train = pd.read_csv('train.csv')
train.isnull().sum()

order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                157
pickup_time               2421
delivered_time            5218
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders           16948
delivered_orders         17341
cancelled                    0
undelivered_orders       17341
lifetime_order_count        53
reassignment_method     436256
reassignment_reason     436247
reassigned_order        436247
session_time              3675
cancelled_time          444782
dtype: int64

In [12]:
train.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            450000 non-null  object 
 1   order_id              450000 non-null  int64  
 2   order_date            450000 non-null  object 
 3   allot_time            450000 non-null  object 
 4   accept_time           449843 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              450000 non-null  int64  
 8   first_mile_distance   450000 non-null  float64
 9   last_mile_distance    450000 non-null  float64
 10  alloted_orders        433052 non-null  float64
 11  delivered_orders      432659 non-null  float64
 12  cancelled             450000 non-null  int64  
 13  undelivered_orders    432659 non-null  float64
 14  lifetime_order_count  449947 non-null  float64
 15  

## Preprocessing

In [14]:
train.order_time.sort_values()

0         2021-01-26 02:21:35
1         2021-01-26 02:33:16
2         2021-01-26 02:39:49
3         2021-01-26 02:47:53
4         2021-01-26 03:06:30
                 ...         
449995    2021-02-06 10:03:16
449996    2021-02-06 10:03:17
449997    2021-02-06 10:03:18
449998    2021-02-06 10:03:19
449999    2021-02-06 10:03:24
Name: order_time, Length: 450000, dtype: object

In [15]:
train.order_id.value_counts() #one duplicate row

181402    2
118842    1
219966    1
439171    1
441218    1
         ..
437375    1
435326    1
441469    1
439420    1
118726    1
Name: order_id, Length: 449999, dtype: int64

In [16]:
train[train.order_id==181402] #discover the index of duplicate rows

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
420455,2021-02-05 15:06:30,181402,2021-02-05 00:00:00,2021-02-05 16:32:56,2021-02-05 16:33:16,NaN,NaN,14538,2.6168,1.24,72.0,72.0,1,0.0,244.0,auto,Reassign,1.0,96.5,2021-02-05 16:59:52
420456,2021-02-05 15:06:30,181402,2021-02-05 00:00:00,2021-02-05 16:32:56,2021-02-05 16:33:16,NaN,NaN,14538,2.6168,1.24,72.0,72.0,1,0.0,244.0,auto,Reassign,1.0,96.5,2021-02-05 16:59:52


In [17]:
train1 = train.drop(420455) # drop duplicate

In [18]:
train1.order_date.value_counts() 

2021-02-05 00:00:00    42340
2021-02-03 00:00:00    41447
2021-01-29 00:00:00    41207
2021-02-02 00:00:00    40875
2021-01-28 00:00:00    40783
2021-02-04 00:00:00    39685
2021-01-31 00:00:00    39254
2021-01-27 00:00:00    39015
2021-01-26 00:00:00    38090
2021-02-01 00:00:00    37833
2021-01-30 00:00:00    37589
2021-02-06 00:00:00    11881
Name: order_date, dtype: int64

In [19]:
train1.allot_time.sort_values()

0         2021-01-26 02:21:59
1         2021-01-26 02:33:57
2         2021-01-26 02:39:57
3         2021-01-26 02:48:25
4         2021-01-26 03:07:21
                 ...         
449716    2021-02-06 10:28:26
449701    2021-02-06 10:35:56
449738    2021-02-06 10:42:09
449095    2021-02-06 10:43:52
448144    2021-02-06 10:52:57
Name: allot_time, Length: 449999, dtype: object

In [20]:
train1.accept_time.isnull().sum() 

157

In [21]:
train1[train1.accept_time.isnull()].cancelled.sum() #checking if ALL null values have cancelled orders

157

In [22]:
train1.pickup_time.isnull().sum()

2420

In [23]:
train1[train1.pickup_time.isnull()].cancelled.sum() #checking if ALL null values have cancelled orders

2420

In [24]:
train1.delivered_time.isnull().sum()

5217

In [25]:
train1[train1.delivered_time.isnull()].cancelled.sum() #checking if ALL null values have cancelled orders

5217

In [26]:
np.sort(train1.rider_id.unique()) #unique rider_id's in train data

array([    0,     1,     2, ..., 21564, 21565, 21566], dtype=int64)

In [27]:
test = pd.read_csv('test.csv')
np.sort(test.rider_id.unique()) #unique rider_id's in test data

array([    0,     1,     2, ..., 13982, 13983, 13984], dtype=int64)

In [28]:
#comparing unique rider_id's in train and test data
riders = train1.rider_id.unique().tolist()
unmatched = False
list1 = []
for i in test.rider_id.unique().tolist():
    if i not in riders:
        list1.append(i)

In [29]:
len(list1) #2030 rider_id';s in test data which are not in train data

2030

In [30]:
train1.first_mile_distance.describe()

count    449999.000000
mean          1.229886
std           0.846181
min           0.000134
25%           0.539550
50%           1.138700
75%           1.853000
max          42.038100
Name: first_mile_distance, dtype: float64

In [31]:
train1.last_mile_distance.describe()

count    449999.000000
mean          2.968877
std           1.884124
min           0.000000
25%           1.470000
50%           2.670000
75%           4.220000
max          22.410000
Name: last_mile_distance, dtype: float64

In [32]:
train1.alloted_orders.isnull().sum()

16948

In [33]:
train2 = train1.copy()
train2.alloted_orders.fillna(0, inplace=True)
train2.delivered_orders.fillna(0, inplace=True) #when alloted and delivered orders are missing, there is no previosuly available data for that rider, so we replace it with 0.

In [34]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449999 entries, 0 to 449999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            449999 non-null  object 
 1   order_id              449999 non-null  int64  
 2   order_date            449999 non-null  object 
 3   allot_time            449999 non-null  object 
 4   accept_time           449842 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              449999 non-null  int64  
 8   first_mile_distance   449999 non-null  float64
 9   last_mile_distance    449999 non-null  float64
 10  alloted_orders        449999 non-null  float64
 11  delivered_orders      449999 non-null  float64
 12  cancelled             449999 non-null  int64  
 13  undelivered_orders    432658 non-null  float64
 14  lifetime_order_count  449946 non-null  float64
 15  

In [35]:
train2.cancelled.value_counts() # imbalanced dataset

0    444782
1      5217
Name: cancelled, dtype: int64

In [36]:
train2.undelivered_orders.value_counts() 

0.0    232685
1.0    118103
2.0     50582
3.0     20011
4.0      7733
5.0      1851
6.0       932
7.0       600
8.0       105
9.0        56
Name: undelivered_orders, dtype: int64

In [37]:
train2.undelivered_orders.isnull().sum()

17341

In [38]:
train2 = train2.assign(undelivered_orders = lambda x: (train2['alloted_orders'] - train2['delivered_orders']))
#replacing missing values in undelivered orders with alloted_orders - delivered_orders

In [39]:
train2.lifetime_order_count.isnull().sum()

53

In [40]:
train3 = train2.copy()
train3.lifetime_order_count.fillna(0,inplace=True) #fill missing values for lifetime_order_count with 0.

In [41]:
train3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449999 entries, 0 to 449999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            449999 non-null  object 
 1   order_id              449999 non-null  int64  
 2   order_date            449999 non-null  object 
 3   allot_time            449999 non-null  object 
 4   accept_time           449842 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              449999 non-null  int64  
 8   first_mile_distance   449999 non-null  float64
 9   last_mile_distance    449999 non-null  float64
 10  alloted_orders        449999 non-null  float64
 11  delivered_orders      449999 non-null  float64
 12  cancelled             449999 non-null  int64  
 13  undelivered_orders    449999 non-null  float64
 14  lifetime_order_count  449999 non-null  float64
 15  

In [42]:
train3.reassigned_order.fillna(0,inplace=True) #0 if order was not reassigned
train3.reassignment_method.fillna(train3.reassignment_method.mode()[0],inplace=True) #replacing nulls with most frequent value
train3.reassignment_reason.fillna(train3.reassignment_reason.mode()[0],inplace=True)  #replacing nulls with most frequent value

In [43]:
train3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449999 entries, 0 to 449999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            449999 non-null  object 
 1   order_id              449999 non-null  int64  
 2   order_date            449999 non-null  object 
 3   allot_time            449999 non-null  object 
 4   accept_time           449842 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              449999 non-null  int64  
 8   first_mile_distance   449999 non-null  float64
 9   last_mile_distance    449999 non-null  float64
 10  alloted_orders        449999 non-null  float64
 11  delivered_orders      449999 non-null  float64
 12  cancelled             449999 non-null  int64  
 13  undelivered_orders    449999 non-null  float64
 14  lifetime_order_count  449999 non-null  float64
 15  

In [44]:
train3.session_time.fillna(0,inplace=True) #repalcing nulls with 0

In [45]:
train4 = train3.copy()
train4.drop('cancelled_time',axis=1, inplace=True) #cancelled_time is not relevant for predicting if an order will be cancelled or not

In [46]:
train4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449999 entries, 0 to 449999
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            449999 non-null  object 
 1   order_id              449999 non-null  int64  
 2   order_date            449999 non-null  object 
 3   allot_time            449999 non-null  object 
 4   accept_time           449842 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              449999 non-null  int64  
 8   first_mile_distance   449999 non-null  float64
 9   last_mile_distance    449999 non-null  float64
 10  alloted_orders        449999 non-null  float64
 11  delivered_orders      449999 non-null  float64
 12  cancelled             449999 non-null  int64  
 13  undelivered_orders    449999 non-null  float64
 14  lifetime_order_count  449999 non-null  float64
 15  

In [38]:
#dropping irrelevant columns
train4.drop('order_id',axis=1,inplace=True)
train4.drop('rider_id',axis=1,inplace=True)
train4.drop('order_date',axis=1,inplace=True)

In [39]:
train4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449999 entries, 0 to 449999
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            449999 non-null  object 
 1   allot_time            449999 non-null  object 
 2   accept_time           449842 non-null  object 
 3   pickup_time           447579 non-null  object 
 4   delivered_time        444782 non-null  object 
 5   first_mile_distance   449999 non-null  float64
 6   last_mile_distance    449999 non-null  float64
 7   alloted_orders        449999 non-null  float64
 8   delivered_orders      449999 non-null  float64
 9   cancelled             449999 non-null  int64  
 10  undelivered_orders    449999 non-null  float64
 11  lifetime_order_count  449999 non-null  float64
 12  reassignment_method   449999 non-null  object 
 13  reassignment_reason   449999 non-null  object 
 14  reassigned_order      449999 non-null  float64
 15  

In [40]:
# Converting the time columns pd.datetime64 datatype
train4['allot_time'] = train4['allot_time'].astype('datetime64[ns]')
train4['order_time']= train4['order_time'].astype('datetime64[ns]')
train4['accept_time'] = train4['accept_time'].astype('datetime64[ns]')

In [41]:
#calculating time delta between allot_time and order_time - this will be our new column allot_delay.
from tqdm import tqdm
t = []
for i in tqdm(range(train4.shape[0])):
    t.append(((train4.iloc[i].allot_time - train4.iloc[i].order_time).seconds))

100%|████████████████████████████████████████████████████████████████████████| 449999/449999 [04:01<00:00, 1864.22it/s]


In [42]:
train4['allot_delay'] = t

In [43]:
#calculating time delta between accept_time and allot_time - this will be our new column accept_delay.
t = []
for i in tqdm(range(train4.shape[0])):
    t.append(((train4.iloc[i].accept_time - train4.iloc[i].allot_time).seconds))

100%|████████████████████████████████████████████████████████████████████████| 449999/449999 [03:55<00:00, 1912.41it/s]


In [44]:
train4['accept_delay'] = t

In [45]:
#there are null values in accept_delay due to null values in accept_time. For now, we are replacing them with zero.
train4.accept_delay.fillna(0,inplace=True)

In [46]:
train5 = train4.copy()
train5.drop(['pickup_time','delivered_time','allot_time','order_time','accept_time'],axis=1,inplace=True)
#dropping all time-related columns - some have been used to calculate time delta so they are no longer needed, others are missing in test data.

In [47]:
train5['delivered_ratio'] = train5.delivered_orders/train5.alloted_orders
#a ratio of delivered to alloted orders is being used instead of the two columns

In [48]:
train5.delivered_ratio.fillna(0,inplace=True)
#there are missing values in the ratio when alloted orders = 0. in such cases, the ratio can be said to be 0.

In [49]:
train5.drop(['alloted_orders','delivered_orders'],axis=1,inplace=True)
#dropping the columns which were used for the delivered_ratio

In [50]:
train6 = pd.get_dummies(train5,columns=['reassignment_method','reassignment_reason'])
#one-hot-encoding the two categorical columns

In [51]:
train6=train6.rename(columns=
                     {'reassignment_reason_Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket':'reassignment_reason_auto'})
#just renaming an unecessarily long column name

In [52]:
train6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449999 entries, 0 to 449999
Data columns (total 15 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   first_mile_distance                                       449999 non-null  float64
 1   last_mile_distance                                        449999 non-null  float64
 2   cancelled                                                 449999 non-null  int64  
 3   undelivered_orders                                        449999 non-null  float64
 4   lifetime_order_count                                      449999 non-null  float64
 5   reassigned_order                                          449999 non-null  float64
 6   session_time                                              449999 non-null  float64
 7   allot_delay                                               449999 non-null  int64  
 8   acce

In [168]:
test = pd.read_csv('test.csv')
test.drop(['order_id','order_date','rider_id'],axis=1,inplace=True)

In [169]:
test.order_time = test.order_time.astype('datetime64[ns]')
test.allot_time = test.allot_time.astype('datetime64[ns]')
test.accept_time = test.accept_time.astype('datetime64[ns]')

In [170]:
from tqdm import tqdm
t = []
for i in tqdm(range(test.shape[0])):
    t.append(((test.iloc[i].allot_time - test.iloc[i].order_time).seconds))

100%|████████████████████████████████████████████████████████████████████████| 144844/144844 [01:06<00:00, 2187.41it/s]


In [171]:
u = []
for i in tqdm(range(test.shape[0])):
    u.append(((test.iloc[i].accept_time - test.iloc[i].allot_time).seconds))

100%|████████████████████████████████████████████████████████████████████████| 144844/144844 [01:06<00:00, 2177.70it/s]


In [172]:
test['allot_delay'] = t

In [173]:
test['accept_delay'] = u

In [174]:
test['delivered_ratio'] = test.delivered_orders/test.alloted_orders
test.delivered_ratio.fillna(0,inplace=True)

In [175]:
test.delivered_orders.fillna(0,inplace=True)
test.undelivered_orders.fillna(test.alloted_orders-test.delivered_orders,inplace=True)

In [176]:
test.undelivered_orders.fillna(0,inplace=True)

In [177]:
test.reassigned_order.fillna(0,inplace=True) #0 if order was not reassigned
test.reassignment_method.fillna(train3.reassignment_method.mode()[0],inplace=True) #replacing nulls with most frequent value
test.reassignment_reason.fillna(train3.reassignment_reason.mode()[0],inplace=True)  #replacing nulls with most frequent value

In [180]:
test.drop(['order_time','allot_time','accept_time','delivered_orders','alloted_orders'],axis=1,inplace=True)

In [181]:
test = pd.get_dummies(test,columns=['reassignment_method','reassignment_reason'])

In [182]:
test.rename(columns={'reassignment_reason_Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket':'reassignment_reason_auto'},inplace=True)

In [187]:
test.lifetime_order_count.fillna(0,inplace=True)
test.session_time.fillna(0,inplace=True)
test.accept_delay.fillna(0,inplace=True)

In [189]:
test.to_csv('Preprocessed_test.csv',index=False)

In [2]:
train6 = pd.read_csv("Preprocessed_train.csv",index_col=0) #This file contains the preprocessed data
test = pd.read_csv("Preprocessed_test.csv")

## Train-Test split

In [4]:
from sklearn.model_selection import train_test_split
X = train6.drop('cancelled',axis=1)
y = train6[['cancelled']]
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, stratify=y)

## Evaluating various basic models

The following two cells together take several hours to run, so they have been commented out and the results have been listed in a markdown cell subsequently.

In [54]:
# import warnings
# warnings.filterwarnings('ignore')
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.ensemble import RandomForestClassifier,VotingClassifier,AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier,ExtraTreesClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.svm import SVC 
# from sklearn.naive_bayes import GaussianNB
# import xgboost as xgb
# from sklearn import model_selection
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import log_loss,roc_auc_score,precision_score,f1_score,recall_score,roc_curve,auc
# from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,fbeta_score,matthews_corrcoef
# from sklearn import metrics
# from sklearn.model_selection import StratifiedKFold
# # function initializing baseline machine learning models
# def GetBasedModel():
#     basedModels = []
#     basedModels.append(('LR_L2'   , LogisticRegression(penalty='l2')))
#     basedModels.append(('LDA'  , LinearDiscriminantAnalysis()))
#     basedModels.append(('KNN7'  , KNeighborsClassifier(7)))
#     basedModels.append(('KNN5'  , KNeighborsClassifier(5)))
#     basedModels.append(('KNN9'  , KNeighborsClassifier(9)))
#     basedModels.append(('KNN11'  , KNeighborsClassifier(11)))
#     basedModels.append(('CART' , DecisionTreeClassifier()))
#     basedModels.append(('NB'   , GaussianNB()))
#     basedModels.append(('SVM Linear'  , SVC(kernel='linear',gamma='auto',probability=True)))
#     basedModels.append(('SVM RBF'  , SVC(kernel='rbf',gamma='auto',probability=True)))
#     basedModels.append(('AB'   , AdaBoostClassifier()))
#     basedModels.append(('GBM'  , GradientBoostingClassifier(n_estimators=100,max_features='sqrt')))
#     basedModels.append(('RF_Ent100'   , RandomForestClassifier(criterion='entropy',n_estimators=100)))
#     basedModels.append(('RF_Gini100'   , RandomForestClassifier(criterion='gini',n_estimators=100)))
#     basedModels.append(('ET100'   , ExtraTreesClassifier(n_estimators= 100)))
#     basedModels.append(('ET500'   , ExtraTreesClassifier(n_estimators= 500)))
#     basedModels.append(('MLP', MLPClassifier()))
#     basedModels.append(('SGD3000', SGDClassifier(max_iter=1000, tol=1e-4)))
#     basedModels.append(('XGB_2000', xgb.sklearn.XGBClassifier(n_estimators= 2000)))
#     basedModels.append(('XGB_500', xgb.sklearn.XGBClassifier(n_estimators= 500)))
#     basedModels.append(('XGB_100', xgb.sklearn.XGBClassifier(n_estimators= 100)))
#     basedModels.append(('XGB_1000', xgb.sklearn.XGBClassifier(n_estimators= 1000)))
#     basedModels.append(('ET1000'   , ExtraTreesClassifier(n_estimators= 1000)))
    
#     return basedModels

# # function for performing 10-fold cross validation of all the baseline models
# def BasedLine2(X_train, y_train,models):
#     # Test options and evaluation metric
#     num_folds = 10
#     scoring = 'recall'
#     results = []
#     names = []
#     for name, model in models:
#         kfold = model_selection.KFold(n_splits=10)
#         cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
#         results.append(cv_results)
#         names.append(name)
#         msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#         print(msg)
         
        
#     return results,msg

In [34]:
# modeclassification_reporttBasedModel()
# names,results = BasedLine2(X_train, y_train,models)

## Precision Scores
- LR_L2: 0.519203 (0.111816)
- LDA: 0.182667 (0.026256)
- KNN7: 0.432639 (0.104596)
- KNN5: 0.351383 (0.114681)
- KNN9: 0.427019 (0.129902)
- KNN11: 0.460542 (0.125037)
- CART: 0.122109 (0.016014)
- NB: 0.058421 (0.006606)
- AB: 0.668102 (0.107318)
- GBM: 0.724387 (0.079425)
- RF_Ent100: 0.810095 (0.065109)
- RF_Gini100: 0.780269 (0.069372)
- ET100: 0.597972 (0.104945)
- ET500: 0.612026 (0.112296)
- MLP: 0.570092 (0.062656)
- SGD3000: 0.156812 (0.198169)

## Recall Scores
- LR_L2: 0.023354 (0.006576)
- AB: 0.058246 (0.011715)
- GBM: 0.071433 (0.012711)
- RF_Ent100: 0.073517 (0.010757)
- ET100: 0.041300 (0.009057)
- MLP: 0.038299 (0.010671)
- SGD3000: 0.039039 (0.044570)
- XGB_500: 0.097123 (0.012894)
- XGB_1000: 0.095503 (0.008709)

#### Algorithms we will consider:
- Logistic Regression
- LDA
- AdaBoost
- Gradient Boosting
- RandomForest
- MLP
- XGBoost

#### Undersampling
- Logreg
- LDA
- KNN7
- Knn9
- KNN11
- CART
- AB
- GBM
- Randomforest
- Extratrees
- MLP
- XGB

#### SMOTE
- Logistic
- LDA
- AB
- GBM

In [56]:
#train6.to_csv('Preprocessed_train.csv')

In [197]:
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_jobs=-1)
brfc.fit(X_train,np.array(y_train['cancelled'].ravel()))

BalancedRandomForestClassifier(n_jobs=-1)

In [198]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, brfc.predict(X_test))

0.6868656720407356

In [216]:
from imblearn.over_sampling import ADASYN
sampler = ADASYN()
X_over, y_over = sampler.fit_resample(X_train, y_train)

In [74]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_over, np.array(y_over).ravel())

LogisticRegression()

In [75]:
roc_auc_score(y_test, logreg.predict(X_test))

0.6181313742122663

In [76]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1, criterion='entropy',max_depth=7)
rfc.fit(X_over,np.array(y_over).ravel())

RandomForestClassifier(criterion='entropy', max_depth=7, n_jobs=-1)

In [77]:
roc_auc_score(y_test, rfc.predict(X_test))

0.625063521107632

In [78]:
from imblearn.ensemble import RUSBoostClassifier
rusbc = RUSBoostClassifier(n_estimators=100, learning_rate=0.1)
rusbc.fit(X_train,np.array(y_train).ravel())

RUSBoostClassifier(learning_rate=0.1, n_estimators=100)

In [79]:
roc_auc_score(y_test, rusbc.predict(X_test))

0.683663606807305

In [203]:
np.array(raw_test.order_id)

array([130231, 130232, 130233, ...,  41186,  41187,  41188], dtype=int64)

In [206]:
raw_test.order_id

0         130231
1         130232
2         130233
3         130234
4         130235
           ...  
144839     41184
144840     41185
144841     41186
144842     41187
144843     41188
Name: order_id, Length: 144844, dtype: int64

In [ ]:
import xgboost as xgb
xgboost = xgb.sklearn.XGBClassifier(n_estimators=100,booster='dart',n_jobs=-1)
xgboost.fit(X_over,y_over)

In [223]:
roc_auc_score(y_test,xgboost.predict(X_test))

0.5751827579423116

In [ ]:
xgrf = xgb.sklearn.XGBRFClassifier()
xgrf.fit(X_over,y_over)

In [221]:
roc_auc_score(y_test, xgrf.predict(X_test))

0.6171754414273065

In [460]:
from imblearn.under_sampling import RandomUnderSampler
X_rus, y_rus = RandomUnderSampler(sampling_strategy=1).fit_resample(X_train, y_train)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier(max_features='sqrt',n_estimators=100)
gbm.fit(X_rus,y_rus)

In [227]:
roc_auc_score(y_test,gbm.predict(X_test))

0.6838053312646308

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100,learning_rate=0.1)
ada.fit(X_rus,y_rus)

In [308]:
roc_auc_score(y_test,ada.predict(X_test))

0.6865035495889722

In [ ]:
ada2 = AdaBoostClassifier(n_estimators=100,learning_rate=0.1)
ada2.fit(X_over,y_over)

In [310]:
roc_auc_score(y_test,ada2.predict(X_test))

0.6483335086723738

In [495]:
y_rus = np.array(y_rus).ravel()
y_train = np.array(y_train).ravel()
y_over = np.array(y_over).ravel()

#### Algorithms we will consider:
- Logistic Regression
- LDA
- AdaBoost
- Gradient Boosting
- RandomForest
- MLP
- XGBoost

# Logistic Regression

In [470]:
from sklearn.model_selection import GridSearchCV
logreg = LogisticRegression(solver='newton-cg',max_iter=200)
logreg.fit(X_rus,y_rus)
roc_auc_score(y_test,logreg.predict(X_test))

0.6537585661276596

Best ROC_AUC_SCORE for undersampled Logreg: 0.6518772452257546

# LDA

In [490]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_rus,y_rus)
roc_auc_score(y_test,lda.predict(X_test)) 

0.6506587457753593

# AdaBoost

In [492]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100,algorithm='SAMME')
ada.fit(X_rus,y_rus)
roc_auc_score(y_test,ada.predict(X_test))

0.6836553347421316

# GradientBoosting

In [493]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_rus,y_rus)
roc_auc_score(y_test,gbc.predict(X_test))

0.6899857441330499

# Random Forest

In [496]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train)
roc_auc_score(y_test,rfc.predict(X_test))

0.5415829602829535

In [497]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_rus,y_rus)
roc_auc_score(y_test,rfc.predict(X_test))

0.6755072912676923

In [498]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_over,y_over)
roc_auc_score(y_test,rfc.predict(X_test))

0.5695793633842354

# MLP

In [499]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train,y_train)
roc_auc_score(y_test,mlp.predict(X_test))

0.5123741149307909

In [500]:
mlp = MLPClassifier()
mlp.fit(X_rus,y_rus)
roc_auc_score(y_test,mlp.predict(X_test))

0.6524606225177945

In [501]:
mlp = MLPClassifier()
mlp.fit(X_over,y_over)
roc_auc_score(y_test,mlp.predict(X_test))

0.6437041214963857

# XGBoost

In [ ]:
import xgboost as xgb
xgboost = xgb.sklearn.XGBClassifier()
xgboost.fit(X_train,y_train)
roc_auc_score(y_test,xgboost.predict(X_test))

In [ ]:
xgboost = xgb.sklearn.XGBClassifier()
xgboost.fit(X_rus,y_rus)
roc_auc_score(y_test,xgboost.predict(X_test))

In [ ]:
xgboost = xgb.sklearn.XGBClassifier()
xgboost.fit(X_over,y_over)
roc_auc_score(y_test,xgboost.predict(X_test))

# Code for creating a submission file

In [208]:
cancelled=pd.Series(brfc.predict(test))

In [200]:
raw_test = pd.read_csv('test.csv')

In [211]:
pd.DataFrame({'order_id':raw_test.order_id,'cancelled':cancelled}).to_csv("submissions1.csv",index=False)

In [52]:
train4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449999 entries, 0 to 449999
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            449999 non-null  object 
 1   order_id              449999 non-null  int64  
 2   order_date            449999 non-null  object 
 3   allot_time            449999 non-null  object 
 4   accept_time           449842 non-null  object 
 5   rider_id              449999 non-null  int64  
 6   first_mile_distance   449999 non-null  float64
 7   last_mile_distance    449999 non-null  float64
 8   alloted_orders        449999 non-null  float64
 9   delivered_orders      449999 non-null  float64
 10  cancelled             449999 non-null  int64  
 11  undelivered_orders    449999 non-null  float64
 12  lifetime_order_count  449999 non-null  float64
 13  session_time          449999 non-null  float64
dtypes: float64(7), int64(3), object(4)
memory usage: 51.

In [49]:
train4.drop(['pickup_time','delivered_time'],axis=1,inplace=True)

In [51]:
train4.drop(['reassignment_method','reassignment_reason','reassigned_order'],axis=1,inplace=True)

In [53]:
train4.drop('order_date',axis=1,inplace=True)

In [54]:
train4.to_csv('train2.csv',index=False)

In [55]:
train5 = train4.dropna()

In [61]:
train5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449842 entries, 0 to 449999
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            449842 non-null  object 
 1   allot_time            449842 non-null  object 
 2   accept_time           449842 non-null  object 
 3   rider_id              449842 non-null  int64  
 4   first_mile_distance   449842 non-null  float64
 5   last_mile_distance    449842 non-null  float64
 6   alloted_orders        449842 non-null  float64
 7   delivered_orders      449842 non-null  float64
 8   cancelled             449842 non-null  int64  
 9   undelivered_orders    449842 non-null  float64
 10  lifetime_order_count  449842 non-null  float64
 11  session_time          449842 non-null  float64
dtypes: float64(7), int64(2), object(3)
memory usage: 44.6+ MB


In [60]:
#train5.drop('order_id',axis=1,inplace=True)

In [5]:
import lightgbm
lgbm = lightgbm.LGBMClassifier(boosting_type='goss',n_estimators=240,max_depth=4,num_leaves=10,class_weight='balanced')#,subsample=0.5,subsample_freq=7)
lgbm.fit(X_train,np.array(y_train).ravel())

LGBMClassifier(boosting_type='goss', class_weight='balanced', max_depth=4,
               n_estimators=240, num_leaves=10)

In [7]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,lgbm.predict(X_test))

0.6862090586798316

In [280]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
under = RandomUnderSampler()
over = RandomOverSampler(sampling_strategy=0.7)
X_over, y_over = over.fit_resample(X_train,y_train)
X_under,y_under = under.fit_resample(X_over,y_over)

In [281]:
lgbm = lightgbm.LGBMClassifier(boosting_type='gbdt',n_estimators=70,max_depth=4,num_leaves=5,subsample=0.45,subsample_freq=10,min_child_samples=500,)
lgbm.fit(X_under,np.array(y_under).ravel())

LGBMClassifier(max_depth=4, min_child_samples=500, n_estimators=70,
               num_leaves=5, subsample=0.45, subsample_freq=10)

In [282]:
roc_auc_score(y_test,lgbm.predict(X_test))

0.6840289680285598

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier()
gbm.fit(X_under,y_under)

In [284]:
roc_auc_score(y_test,gbm.predict(X_test))

0.6860791953400606